In [1]:
spark

In [1]:
from pyspark.sql import SparkSession
#initialize spark session
spark=SparkSession.builder \
.appName("ReduceByKey_vs_GroupByKey")\
.getOrCreate()
         

25/04/13 11:09:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
!hadoop fs -ls /user/msd24002

Found 7 items
drwxr-xr-x   - msd24002 hadoop          0 2025-04-07 14:39 /user/msd24002/.sparkStaging
-rw-r--r--   2 msd24002 hadoop    1060750 2025-04-13 10:57 /user/msd24002/customers1mb.csv
drwxr-xr-x   - msd24002 hadoop          0 2025-03-30 14:52 /user/msd24002/input
-rw-r--r--   2 msd24002 hadoop        141 2025-04-07 14:04 /user/msd24002/input.txt
drwxr-xr-x   - msd24002 hadoop          0 2025-04-07 14:39 /user/msd24002/output_two_parts
drwxr-xr-x   - msd24002 hadoop          0 2025-03-30 14:54 /user/msd24002/output_two_reducers
drwxr-xr-x   - msd24002 hadoop          0 2025-04-07 14:18 /user/msd24002/output_wordcount


In [3]:
!hadoop fs -head /user/msd24002/customers1mb.csv



customer_id,name,city,state,country,registration_date,is_active
0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False
5,Customer_5,Hyderabad,Karnataka,India,2023-07-28,False
6,Customer_6,Pune,Delhi,India,2023-08-29,False
7,Customer_7,Ahmedabad,West Bengal,India,2023-12-28,True
8,Customer_8,Pune,Karnataka,India,2023-06-22,True
9,Customer_9,Mumbai,Telangana,India,2023-01-05,True
10,Customer_10,Pune,Gujarat,India,2023-08-05,True
11,Customer_11,Delhi,West Bengal,India,2023-08-02,False
12,Customer_12,Chennai,Gujarat,India,2023-11-21,False
13,Customer_13,Chennai,Karnataka,India,2023-11-06,True
14,Customer_14,Hyderabad,Tamil Nadu,India,2023-02-07,False
15,Customer_15,Mumbai,Gujarat,India,2023-03-02,True
16,Customer_16,Chennai,Karnataka,India,2023-04-05,False
17,Customer_17,Hyd

In [4]:
hdfs_path='/user/msd24002/customers1mb.csv'
rdd=spark.sparkContext.textFile(hdfs_path)

In [5]:
header=rdd.first()

In [6]:
rdd_no_header=rdd.filter(lambda row: row!=header).map(lambda row:row.split(','))

In [7]:
rdd_no_header.first()

['0', 'Customer_0', 'Pune', 'Maharashtra', 'India', '2023-06-29', 'False']

In [10]:
reduced_rdd=rdd_no_header.map(lambda row:(row[2],1)).reduceByKey(lambda x,y:x+y)

In [11]:
reduced_rdd.collect()

[('Pune', 2243),
 ('Hyderabad', 2242),
 ('Mumbai', 2142),
 ('Delhi', 2200),
 ('Bangalore', 2211),
 ('Ahmedabad', 2198),
 ('Chennai', 2194),
 ('Kolkata', 2223)]

In [12]:
grouped_by_rdd=rdd_no_header.map(lambda row:(row[2],1)).groupByKey()

In [13]:
grouped_by_rdd.collect()

[('Pune', <pyspark.resultiterable.ResultIterable at 0x7ff50d70d410>),
 ('Hyderabad', <pyspark.resultiterable.ResultIterable at 0x7ff50d703490>),
 ('Mumbai', <pyspark.resultiterable.ResultIterable at 0x7ff50c65d690>),
 ('Delhi', <pyspark.resultiterable.ResultIterable at 0x7ff50c65d5d0>),
 ('Bangalore', <pyspark.resultiterable.ResultIterable at 0x7ff50c65d790>),
 ('Ahmedabad', <pyspark.resultiterable.ResultIterable at 0x7ff50c65d850>),
 ('Chennai', <pyspark.resultiterable.ResultIterable at 0x7ff50c65d950>),
 ('Kolkata', <pyspark.resultiterable.ResultIterable at 0x7ff50c65d890>)]

In [15]:
grouped_by_result=grouped_by_rdd.map(lambda row :(row[0],len(row[1])))

In [18]:
grouped_by_result.collect()

[('Pune', 2243),
 ('Hyderabad', 2242),
 ('Mumbai', 2142),
 ('Delhi', 2200),
 ('Bangalore', 2211),
 ('Ahmedabad', 2198),
 ('Chennai', 2194),
 ('Kolkata', 2223)]